## 介绍

数据集有15000张图片，每张图片大小为64*64

![](https://img-blog.csdnimg.cn/img_convert/a3603def0440e670ab53aa51474d708e.png)

csv文件内容由以下标签组成：
- suite_id
- sample_id
- code
- value
- character

![](https://img-blog.csdnimg.cn/img_convert/d5230cf6b0cdf28eb737b45c30f6e981.png)

结合以上信息我们可以得到文件名和标签的对应关系: `input_%suite_id%_%sample_id%_%code%.jpg`

## 准备

### 加载项目需要使用的库

In [ ]:
import os
import shutil
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
#import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

### 设定工作目录

In [ ]:
os.chdir(os.path.dirname(os.path.abspath('__file__')))

### 设定文件路径

In [ ]:
DatasetArchieve_Path = './Dataset.zip' # 数据文件压缩包路径
Dataset_Dir = './data' # 数据文件解压目录
MINSTCSV_Path = './chinese_mnist.csv' # csv文件路径
SaveModel_Dir = './model' # 模型保存目录
PredictPic_Path = './predict.jpg' # 预测文件路径

### 创建文件夹

In [ ]:
#os.makedirs(Dataset_Dir, exist_ok = True)
os.makedirs(SaveModel_Dir, exist_ok = True)

### 解压数据文件

In [ ]:
shutil.unpack_archive(
    filename = DatasetArchieve_Path,
    extract_dir = Dataset_Dir,
    format = DatasetArchieve_Path.rsplit('.',1)[-1]
)

## 模型构建

### 全连接神经网络模型（废案）

Q：为啥全连接层神经网络无法很好的处理图像数据？<br>
A：使用全连接神经网络处理图像的最大问题就是：全连接层的参数太多，若数据集每一张图片的大小是28 * 28 * 1。假设第一层隐藏层的节点数为500个，那么一个全连接层的神经网络有28 * 28 * 500 + 500 = 392500个参数，而且有的图片会更大或者是彩色的图片，这时候参数将会更多。参数增多除了导致计算速度减慢，还很容易导致过拟合的问题。所以需要一个合理的神经网络结构来有效的减少神经网络中参数的个数。卷积神经网络就可以更好 的达到这个目的

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(4096, 2048) # 因为每张图片的尺寸都是64，所以初始输入就是4096个特征
        self.l2 = torch.nn.Linear(2048, 2048)
        self.l3 = torch.nn.Linear(2048, 1024)
        self.l4 = torch.nn.Linear(1024, 1024)
        self.l5 = torch.nn.Linear(1024, 128)
        self.l6 = torch.nn.Linear(128, 15)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.norm = torch.nn.BatchNorm1d(128)

    def forward(self, x):
        x = x.view(-1, 4096)  # view()目的是对tensor进行reshape，将向量铺平。输入数据被解析为4096维（列数）的特征，方便后续计算以及传入全连接层，-1表示自动计算（行数）
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x)) + x
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x)) + x
        x = F.relu(self.norm(self.l5(x)))
        return self.dropout(self.l6(x))  # 最后一层不做激活

### 卷积

**概述**
1. 左边这个二维矩阵表示的是一张6 * 6 * 1的图片，意思是一张长为6宽为6通道数为1的图片（单通道图片一般叫灰度图）
2. 中间是3 * 3 * 1的二维矩阵，在卷积神经网络中称为卷积核或者过滤器，是整个神经网络要训练取得的参数。卷积核会图像矩阵中移动，每次移动的步长可自定
3. 右边的图片是该卷积核移动的结果（一个卷积核产生一个一维的输出图像）。比如从左上角开始计算：3 * 1 + 0 * 0 + 1 * -1 + 1 * 1 + 5 * 0 + 8 * -1 + 2 * 1 + 7 * 0 + 2 * -1 = -5

![](https://img-blog.csdnimg.cn/20201123105410961.png)

假设我们有一张图像是NN的，过滤器FF，最后得到(N-F+1) * (N-F+1)的图像。这样就会有两个缺点：
- 边缘的值因为被计算的次数较少，因此可能会丢失调边缘的特征
- 一张图片经过层层的卷积之后，输出的图片越来越小了（解决方案：padding）

**Padding**<br>
在图像周围用0填充。padding有两种:
- valid模式：输出的图像不需要与原图像大小相等
- same模式：输出的图像需要与原图像大小相等

Q：要在图像周围填充多少层0才能保证输出的图像与原图像等大小呢<br>
A：设加一圈0则P = 1，输出图像大小从(N-F+1) * (N-F+1)变成了(N+2P-F+1) * (N+2P-F+1)。因为要令输入的大小等于输出的大小，所以P = (F-1)/2

**Stride**<br>
输入与输出图像之间的关系公式需要更新为：

![](https://img-blog.csdnimg.cn/20201123152713818.png)

### Relu激活函数

全称为修正线性单元（Rectified Linear Unit）

Relu的特点：
- 当x<=0时，ReLU=0
- 当x>0时，ReLU=x

Relu的导数特点：
- 当x<=0时，ReLU的导数为0
- 当x>0时，ReLU的导数为1

Relu的优点：
- 由Relu的原始图像和导数图像可知，Relu可能使部分神经元的值变为0，可以使神经元“死亡”，降低神经网络复杂性，从而有效缓解过拟合的问题
- 由于当x>0时，Relu的梯度恒为1，所以随着神经网络越来越复杂，不会导致梯度累乘后变得很大或很小，从而不会发生梯度爆炸或梯度消失问题
- Relu的计算非常简单，就是取0或者值本身，提高了神经网络的效率

### 池化层

作用主要是减小模型的大小，提高运算速度

**Max Pooling**<br>
即最大池化。下图这个最大池化使用的是2 * 2的窗口，步长是2，取的是原窗口中最大的那个数。输入图像在输出后高宽减半了

### 卷积神经网络模型

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 定义卷积核为5×5的卷积层1，令通道数从1变为10（输入通道为1代表图片应当是单通道灰度型而非三通道RGB型）
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 定义卷积核为5×5的卷积层2，令通道数从10变为20
        self.pooling = nn.MaxPool2d(2, 2) # 定义卷积核为2×2的最大池化层，步长为2（默认与卷积核大小相同，不设置也一样）
        self.fc = nn.Linear(320, 15) # 定义输入维度为320输出维度为15的全连接层

    def forward(self, x):
        size = x.size(0)
        x = self.pooling(F.relu(self.conv1(x))) # 通过之前定义的卷积层1，然后激活，最后通过最大池化层（高宽变为一半）
        x = self.pooling(F.relu(self.conv2(x))) # 通过之前定义的卷积层2，然后激活，最后通过最大池化层（高宽变为一半）
        x = x.view(size, -1) # view()目的是对tensor进行reshape，将向量铺平。这里输入数据被解析为自动计算列数（即上面定义的320）维的特征，方便后续计算以及传入全连接层
        x = self.fc(x) # 通过全连接层
        return x # 最后输出的是维度为10的，也就是对应数学符号的0~9

### 模型加载并允许GPU训练

In [ ]:
model = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 允许GPU训练
model.to(device) # 加载模型到指定设备

## 数据处理

### 根据模型需求处理并获取图片数据

因为数据集图片的尺寸都是64，所以resize的尺寸也是64*64（推理非数据集图片时会用到）

In [ ]:
def GetImageData(
    ImgPath: str,
    Resize: bool = False,
    Resize_Height: int = 64,
    Resize_Width: int = 64
):
    image_mat = cv2.imread(ImgPath) # 读图（转mat类型）
    if Resize:
        image_mat = cv2.resize(image_mat, (2*Resize_Height, 2*Resize_Width), None) # 尺寸调整
    image_mat = cv2.cvtColor(image_mat, cv2.COLOR_BGR2GRAY) / 255 # 灰度化
    image_mat = 255 - cv2.threshold(image_mat.astype("uint8"), 0, 255, cv2.THRESH_OTSU)[-1] # 二值化
    return image_mat.astype(np.float32) # 防止精度报错：mat1 and mat2 must have the same dtype

### 设定字典

In [ ]:
character_dict = {'零': 0, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '十': 10, '百': 11, '千': 12, '万': 13, '亿': 14}

## 数据加载

### 定义数据集

Dataset是用来创建数据集的函数（自定义数据集的内容和标签），一般情况下，我们的数据集首先传给Dataset，封装为数据集类。

In [ ]:
from torch.utils.data import Dataset

如果我们要自定义一个Dataset供我们使用，通常要完成以下三个函数（也可以不重写，直接调用，也可以返回数据集相应的信息）

In [ ]:
class CN_MNIST(Dataset):
    
    # 初始化数据内容和标签
    def __init__(self, index_csv, character_dict):
        self.index_csv = index_csv
        self.dictionary = character_dict
    
    # 获取数据内容和标签
    def __getitem__(self, index):
        sample = self.index_csv.iloc[index]
        label = self.dictionary[str(sample['character'])]
        suite_id = sample['suite_id']
        sample_id = sample['sample_id']
        code = sample['code']
        image_path = os.path.join(Dataset_Dir, ('input_' + str(suite_id) + '_' + str(sample_id) + '_' + str(code) + '.jpg'))
        image = GetImageData(image_path, False)
        return torch.Tensor(image), torch.Tensor([label]).squeeze().long()
    
    # 获取数据集大小
    def __len__(self):
        return len(self.index_csv['code'])

### 加载数据

Dataloader又称为数据加载类，其作用是接收来自Dataset已经加载好的数据集，对原数据集进行顺序打乱、分批等操作。

In [ ]:
from torch.utils.data import DataLoader

重要参数
- dataset：从Dataset中传入的原始数据集类
- batch_size：每批数据的大小
- shuffle：是否打乱数据集
- num_workers：工作的线程数

In [ ]:
# 批处理量
batch_size = 128
# 训练集
train_dataset = CN_MNIST(pd.read_csv(MINSTCSV_Path).sample(frac=1), character_dict)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=0) # 打乱
# 测试集
test_dataset = CN_MNIST(pd.read_csv(MINSTCSV_Path).sample(frac=1), character_dict)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, num_workers=0) # 不打乱

## CrossEntropyLoss

CrossEntropyLoss是手写数字分类问题的最后一步，其中主要包括三个步骤：
- Softmax
- Log
- NLLLoss

In [ ]:
# 交叉熵损失
criterion = torch.nn.CrossEntropyLoss() # 交叉熵损失相当于Softmax+Log+NllLoss

### Softmax

Softmax回归是一个线性多分类模型，在MINIST手写数字识别问题中，Softmax最终会给出预测值对于10个类别（0~9）出现的概率，最终模型的预测结果就是概率最大的类别

计算公式
>Softmax(zi)=exp(zi)/∑jexp(zj)exp(zi)

- zi：多分类中的第i类的输出值
- 分子：使用指数函数将实数输出映射到零到正无穷
- 分母：将所有类别的输出值求和，使用指数函数来将其转换为概率，最终将神经网络上一层的原始数据归一化到[0,1]

### Log

经过Softmax后，还要将其结果取Log​（对数），目的是将乘法转化为加法，从而减少计算量，同时保证函数的单调性，因为ln(x)单调递增且：
>ln(x)+ln(y)=ln(x×y)

### NLLLoss

最终使用NLLLoss计算损失，损失函数定义为：
>Loss(Y^,Y)=−YlogY^

参数含义：
- Y^表示Softmax经过Log​后的值
- Y为训练数据对应target的One-hot编码，表示此训练数据对应的target。

Ont-hot编码（独热编码）：某个值有效，对应位置就是1，否则就是0，每一个One-hot编码对应一种状态，在手写数字识别中，可以使用One-hot的不同编码来分别对应每一种数字

## 优化器

### SGD（废案）

Q：什么时候选用随机梯度下降算法作为优化器？<br>
A：若样本非常多，在进行参数优化时，如果对所有的样本都求梯度，那么开销会非常大，所以使用随机梯度算法，每次随机采样不同的少量样本进行参数优化，这样可以降低时间复杂度

In [ ]:
# 优化器
optimizer = optim.SGD(model.parameters(), lr = 0.8) # 第一个参数是初始化参数值，第二个参数是学习率

梯度下降算法是为了找到最优的参数值，使得损失函数最小，假设损失函数定义为：
>J(x)= 1/n n∑i=1 J(xi)

对于损失函数J(x)来说，x是变量，要求损失函数J(x)的最小值，应让损失函数J(x)对x求偏导，表示损失函数在x处的梯度：
>∇J(x)= 1/n∇n∑i=1 J(xi)

然后更新x：
>x=x−ε∇J(x)

ε就是我们常说的学习率，通过调整学习率最终得到最优的x，使得：
>∇J(x)=0

此时梯度为0，说明已经达到极值点，也就是损失函数最小的点，此时的x就是我们要求的最优参数

### Adam

Adam参数更新方法效率更高

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)
#scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-9)

## 模型训练

模型训练过程的步骤一般是固定的，主要包括：

1. 获取训练数据以及其对于标签
<br>测试的时候不需要数据对应的标签，但是训练的时候需要数据对应的标签，因为这样才能使用损失函数计算预测值与真实值之间的误差，从而最优化参数

2. 梯度清零
<br>因为训练的时候是分批次的，如果不设置梯度清零，就会将上一批次的梯度累加到当前批次，可能会造成模型预测不准确，所以每个批次都需要设置梯度清零

3. 模型预测
<br>利用之前建立好的模型对输入数据进行预测，得到预测值，其中包括最大预测值与其对应的数字下标

4. 计算损失
<br>利用之前创建好的CrossEntropyLoss计算损失值

5. 反向传播
<br>应用反向传播的目的就是修正损失函数L(z)中的权值w，使L(z)达到最小

6. 更新参数
<br>利用之前定义的随机梯度下降优化器，并结合反向传播得到的参数与学习率更新参数，可以更好的拟合数据，使损失值更小

7. 保存模型
<br>将模型保存为字典形式，随着测试轮数的增加，模型的准确率会越来越高，后续就可以直接使用训练好的模型进行预测

8. 保存优化器
<br>有时候我们还需要优化器的相关内容，所以也将优化器以字典形式保存

9. 打印损失
<br>为了可视化模型的训练准确率，我们还需要打印模型的损失值，随着训练轮数的增加，模型的损失值会越来越低，而准确率会越来越高

In [ ]:
correct_list = []
def train(epoch):
    total = 0
    correct = 0
    running_loss = 0
    for index, data in enumerate(train_loader, 0):
        inputs, target = data  # 输入和标签
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad() # 梯度清零
        outputs = model(inputs) # 模型预测
        loss = criterion(outputs, target) # 计算损失
        loss.backward() # 反向传播
        optimizer.step() # 更新参数
        # scheduler.step()
        if index % 100 == 0: # 每一百次保存一次模型，打印损失
            torch.save(model.state_dict(), os.path.join(SaveModel_Dir, "model.pkl")) # 保存模型
            torch.save(optimizer.state_dict(), os.path.join(SaveModel_Dir, "optimizer.pkl")) # 保存优化器
            running_loss += loss.item()
            predicted = torch.argmax(outputs.data, dim=1)  # 返回最大值下标
            total += target.size(0)
            correct += (predicted == target).sum().item()
            print('[%d] loss:%.3f' % (epoch + 1, running_loss))
            running_loss = 0.0
            rate = 100 * correct / total
            print('训练集的准确率为: {:.2f}'.format(rate))
            correct_list.append(rate)

## 模型评估

具体的步骤如下：

1. 取消梯度计算
<br>因为梯度计算是为了在训练的时候最优化参数的，而测试的时候，我们是利用训练的模型进行预测，所以不需要最优化参数，也就不需要进行梯度计算。在Pytorch中使用torch.no_grad()取消梯度计算

2. 读取测试数据
<br>直接使用for循环从测试数据中读取数据，分别包括：
    - 原始图片
    - 对应的target（真实值标签）

3. 预测分类
<br>直接使用之前训练好的模型进行预测，取出概率值最大的元组，其中分别包括：
    - 最大概率值
    - 最大概率值的下标（对应0~9个数字，哪个位置概率值最大，就说明分类的结果就是对应的数字）

4. 计算准确率
<br>分别求出此批次的总数以及预测成功的个数，用预测成功的个数除以此批次的总数就是本批次预测的准确率，计算后打印即可

In [ ]:
correct_list2 = []
def eval():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取每一行最大值为预测结果
            probability, predicted = torch.max(outputs.data, dim=1) # 返回一个元组，第一个为最大概率值，第二个为最大概率值的下标
            total += labels.size(0) # target是形状为(batch_size,1)的矩阵，使用size(0)取出该批的大小
            correct += (predicted == labels).sum().item() # predict和target均为(batch_size,1)的矩阵，sum()求出相等的个数
        rate = 100 * correct / total
        print('测试集的准确率为: {:.2f}'.format(rate))
        print('-------------------------')
        correct_list2.append(rate)

## 执行 训练+评估 并可视化

### 设定迭代轮数

In [ ]:
epoch_total = 30

### 执行

In [ ]:
# 在每轮迭代中执行训练+评估
for epoch in range(epoch_total):
    # train
    model.train()
    train(epoch)
    # eval
    model.eval()
    eval()

### 结果可视化

In [ ]:
# 绘制Epoch-Loss曲线
plt.figure()
plt.xlabel('Epoch')
plt.ylabel('accuracy%')
plt.plot(np.arange(0, epoch_total, 1), np.array(correct_list))
plt.plot(np.arange(0, epoch_total, 1), np.array(correct_list2))
plt.show()

## 自定义数字识别

### 加载模型

In [ ]:
model.load_state_dict(torch.load(os.path.join(SaveModel_Dir, "model.pkl"))) # 加载保存模型的参数

具体的步骤如下：

1. 取消梯度计算
<br>因为梯度计算是为了在训练的时候最优化参数的，而测试的时候，我们是利用训练的模型进行预测，所以不需要最优化参数，也就不需要进行梯度计算。在Pytorch中使用torch.no_grad()取消梯度计算

2. 读取测试数据
<br>直接读取原始图片

3. 预测分类
<br>直接使用之前训练好的模型进行预测，取出概率值最大的元组，其中分别包括：
    - 最大概率值
    - 最大概率值的下标（对应0~9个数字，哪个位置概率值最大，就说明分类的结果就是对应的数字）

In [ ]:
def predict():
    with torch.no_grad():
        image = torch.tensor(GetImageData(PredictPic_Path, True)).to(device)
        outputs = model(image)
        # 取最大值为预测结果
        probability, predicted = torch.max(outputs.data, dim=1) # 返回一个元组，第一个为最大概率值，第二个为最大概率值的下标
        result = list(character_dict.keys())[predicted[0].item()]
        print(f'图片为: {result}')
        print('-------------------------')

### 执行

In [ ]:
model.eval()
predict()